In [1]:
import pandas as pd
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, exists
from sqlalchemy.sql import func

engine = create_engine('sqlite:///../Data/Students.db')
metadata_obj = MetaData() # Объект-контейнер, содержит информацию о схеме данных
metadata_obj.reflect(bind=engine) # Отображение БД
# Вызов метода reflect() приводит к включению в объект metadata_obj словаря всех таблиц БД.
conn=engine.connect() # Соединение

exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")

In [2]:
#1. Напишите запрос с EXISTS, позволяющий вывести данные обо всех студентах обучающихся в вузах, имеющих рейтинг выше 300.
# SQL - SELECT * FROM student WHERE univ_id IN (SELECT univ_id FROM university WHERE rating > 300)

s = exists(university).where(university.c.rating > 300)

sq = s.where(student.c.univ_id == university.c.univ_id)

s = select(student)

s = s.where(sq)

print(s)
pd.read_sql(s, conn)

SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student 
WHERE EXISTS (SELECT university.univ_id, university.univ_name, university.rating, university.city 
FROM university 
WHERE university.rating > :rating_1 AND student.univ_id = university.univ_id)


,student_id,surname,name,stipend,kurs,city,birthday,univ_id
0,6,Сидоров,Вадим,150,4,Москва,1999-06-07,22
1,15,Ордуханян,Рафаэль,120,1,Липецк,2003-06-01,18
2,32,Котов,Павел,150,5,Белгород,2001-02-28,14
3,57,Кудряшова,Вера,100,5,Нижневартовск,2002-02-18,14
4,58,Журавлева,Вера,0,4,Новосибирск,2003-06-14,11
...,...,...,...,...,...,...,...,...
58,204,Петров,Вадим,250,2,Брянск,2002-05-21,44
59,205,Иванова,Мария,200,1,Липецк,2002-04-11,32
60,206,Киселев,Владимир,210,2,Орел,2003-02-10,44
61,207,Миронова,Федор,200,3,Москва,2002-04-15,45


In [126]:
# 2. Напишите предыдущий запрос, используя соединения.
# SQL - SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id, university.univ_name FROM student JOIN university on student.univ_id = university.univ_id WHERE EXISTS (SELECT university.univ_id, university.univ_name, university.rating, university.city FROM university WHERE university.rating > 300 AND student.univ_id = university.univ_id)

s = select(student, university.c.univ_name).where(university.c.rating > 300).join(university, student.c.univ_id == university.c.univ_id)

print(s)
pd.read_sql(s, conn)

SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id, university.univ_name 
FROM student JOIN university ON student.univ_id = university.univ_id 
WHERE university.rating > :rating_1


,student_id,surname,name,stipend,kurs,city,birthday,univ_id,univ_name
0,6,Сидоров,Вадим,150,4,Москва,1999-06-07,22,МГУ
1,15,Ордуханян,Рафаэль,120,1,Липецк,2003-06-01,18,ВГМА
2,32,Котов,Павел,150,5,Белгород,2001-02-28,14,БГУ
3,57,Кудряшова,Вера,100,5,Нижневартовск,2002-02-18,14,БГУ
4,58,Журавлева,Вера,0,4,Новосибирск,2003-06-14,11,НГУ
...,...,...,...,...,...,...,...,...,...
58,204,Петров,Вадим,250,2,Брянск,2002-05-21,44,ФинУ
59,205,Иванова,Мария,200,1,Липецк,2002-04-11,32,РГУ
60,206,Киселев,Владимир,210,2,Орел,2003-02-10,44,ФинУ
61,207,Миронова,Федор,200,3,Москва,2002-04-15,45,МГТУ им. Баумана


In [3]:
# 3. Напишите запрос с EXISTS, выбирающий сведения о каждом студенте, для которого в городе его проживания имеется хотя бы один университет, в котором он не учится.

s = exists(university).where(university.c.city != student.c.city)

sq = s.where(student.c.univ_id == university.c.univ_id)

s = select(student)

s = s.where(sq)


print(s)
pd.read_sql(s, conn)

SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student 
WHERE EXISTS (SELECT university.univ_id, university.univ_name, university.rating, university.city 
FROM university 
WHERE university.city != student.city AND student.univ_id = university.univ_id)


,student_id,surname,name,stipend,kurs,city,birthday,univ_id
0,1,Иванов,Иван,150,1,Орел,2002-12-03,10
1,3,Петров,Петр,200,3,Курск,2000-12-01,10
2,10,Кузнецов,Борис,0,2,Брянск,2001-12-08,10
3,12,Зайцева,Ольга,250,2,Липецк,2001-05-01,10
4,15,Ордуханян,Рафаэль,120,1,Липецк,2003-06-01,18
...,...,...,...,...,...,...,...,...
74,202,Владимиров,Анатолий,200,2,Курск,2003-05-21,14
75,204,Петров,Вадим,250,2,Брянск,2002-05-21,44
76,205,Иванова,Мария,200,1,Липецк,2002-04-11,32
77,206,Киселев,Владимир,210,2,Орел,2003-02-10,44


In [87]:
# 4. Напишите запрос, выбирающий из таблицы SUBJECT данные о названиях предметов обучения, экзамены по которым сданы более чем одним студентом.

s = exists(exam_marks.c.subj_id).where(exam_marks.c.mark > 2)
s = s.where(subject.c.subj_id == exam_marks.c.subj_id)

sq = select(exam_marks.c.subj_id).where(s).having(func.count(exam_marks.c.student_id) > 1)
sq = sq.group_by(exam_marks.c.subj_id)

sqs = select(subject).where(subject.c.subj_id.in_(sq))

print(sqs)
pd.read_sql(sqs, conn)

SELECT subject.subj_id, subject.subj_name, subject.hour, subject.semester 
FROM subject 
WHERE subject.subj_id IN (SELECT exam_marks.subj_id 
FROM exam_marks 
WHERE EXISTS (SELECT exam_marks.subj_id 
FROM subject 
WHERE exam_marks.mark > :mark_1 AND subject.subj_id = exam_marks.subj_id) GROUP BY exam_marks.subj_id 
HAVING count(exam_marks.student_id) > :count_1)


,subj_id,subj_name,hour,semester
0,10,Информатика,56,1
1,22,Физика,34,1
2,56,История,34,4
3,73,Физкультура,34,5
4,94,Английский язык,56,3


In [140]:
# 5. Напишите запрос для получения списка городов проживания студентов, в которых есть хотя бы один университет.

sq = exists(university).where(university.c.city == student.c.city)

s = select(student)
s = s.where(sq)

print(s)
pd.read_sql(s, conn)

SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student 
WHERE EXISTS (SELECT university.univ_id, university.univ_name, university.rating, university.city 
FROM university 
WHERE university.city = student.city)


,student_id,surname,name,stipend,kurs,city,birthday,univ_id
0,6,Сидоров,Вадим,150,4,Москва,1999-06-07,22
1,32,Котов,Павел,150,5,Белгород,2001-02-28,14
2,55,Белкин,Вадим,250,5,Воронеж,2000-01-07,10
3,58,Журавлева,Вера,0,4,Новосибирск,2003-06-14,11
4,75,Игнатова,Зинаида,100,5,Ростов,2000-09-26,22
5,200,Баранов,Александр,220,2,Москва,2001-08-18,44
6,203,Петрова,Екатерина,250,4,Белгород,2001-05-21,14
7,207,Миронова,Федор,200,3,Москва,2002-04-15,45
8,265,Павлов,Андрей,0,3,Воронеж,2000-11-05,10
9,276,Петров,Антон,200,4,Ульяновск,2001-08-05,22
